In [ ]:
from src import *

## 1. Data acquisition

In [ ]:
# Get data from 2022 to 2023
start_year = 2018
end_year = 2022

nhl_data_provider = get_data_from(start_year, end_year)

In [ ]:
# Display the first 5 row of the regular season data for the 123rd game of the 2022 season
nhl_data_provider.regular_season[2020][123]['plays'][0:5]

## 2. Debugging tools

In [ ]:
#TODO Remplacer homeTeamDefendingSide
season_list = [year for year in range(start_year, end_year + 1)]

ipywidgets.interact(plot_nhl_data, nhl_data_provider=ipywidgets.fixed(nhl_data_provider),
                    game_type=['regular', 'playoff'], season=season_list)

## 3. Data cleaning

In [ ]:
# TODO Changer distance et angle en fonction de zoneShoot
# Clean the data
clean_regular_season, clean_playoff = clean_data(nhl_data_provider)

In [ ]:
# Display the first 5 rows of the cleaned regular season data for the 123rd game of the 2022 season
clean_regular_season[2020][123].head()

## 4. Data analysis

## Get data for question 1

In [ ]:
# Convert regular season and playoff data to dataframes
df_data_2022 = convert_dictionaries_to_dataframes(clean_regular_season, clean_playoff, [2022])
df_data_2022.head()

## Correlation entre but et type de tir

In [ ]:
# Get the correlation between the goal and the type of shot
q = get_correlations_2variables(df_data_2022, 'shotType', 'typeDescKey')
q.head()

## Plot q1

In [ ]:
# Plot the correlation between the goal and the type of shot
plot_correlations_2variables(q, 'Total', 'goal')

## Q1.A : Corrélation entre distance de tir, but, et saison

In [ ]:
# Convert regular season and playoff data to dataframes for the 2018, 2019 and 2020 seasons
df_data_2018_2020 = convert_dictionaries_to_dataframes(clean_regular_season,
                                                       clean_playoff,
                                                       [2018, 2019, 2020])

df_data_2018_2020.head()

In [ ]:
q1a = df_data_2018_2020
print(q1a['shotDistance'].describe())

q1a['shotDistance_Group'] = q1a['shotDistance'].apply(lambda x: 'a.Under 7 ft.' if x < 7 else (
    'b.7 - 14.9 ft.' if x < 15 else (
        'c.15 - 29.9 ft.' if x < 30 else ('d.30 - 44.9 ft.' if x < 45 else 'e.Over 45 ft.'))))

print(q1a['shotDistance_Group'].value_counts())

In [ ]:
# Correlation between goal, shot distance, and season
q1a_r = get_correlations_3variables(q1a, 'shotDistance_Group', 'Year', 'typeDescKey', 'goal')
q1a_r.head()

## Plot Question 1 (Q1)

In [ ]:
# q1a_r",
data_2018 = q1a_r[q1a_r.columns[q1a_r.columns.str.startswith(str(2018))]]

# data_2018
plot_graph_correlations(q1a_r, 'goal', q1a, 'Year')

## Boxplot q1a

In [ ]:
q1a.reset_index(drop=True, inplace=True)

plot_boxplot_correlations(q1a, 'Year', 'shotDistance', 'typeDescKey')

## Q2 : Corrélation entre distance de tir, type de tir, et but

In [ ]:
# shot types
q1a['shotType'].unique()

# Données pour 2020
q2_2020 = q1a[q1a['Year'] == '2020']
q2_2020.head()

# Descriptive stat

In [ ]:
# Descriptive statistics for the shot distance
get_correlations_3variables(q2_2020, 'shotDistance_Group', 'shotType', 'typeDescKey', 'goal')

# Boxplot q2

In [ ]:
# Boxplot for the correlation between the shot distance, the shot type, and the goal
plot_boxplot_correlations(q2_2020, 'shotType', 'shotDistance', 'typeDescKey')

In [ ]:
# TODO remplacer homeTeamDefendingSide
#VISUALISATIONS AVANCÉES
# Plot the shots of the Senators in the 2022 regular season
year = 2020
team_name = 'Senators'
plot_team_shots(clean_regular_season, year, team_name)